In [1]:
# def transistion_policy_solo(f):
#     # print(list(f))
#     assert np.sum(f) <= N # make sure the total quanta do not exceed N
#     if np.isin(N, f): #if one tile has all tiles already
#         next_states = np.where(f < N, 0, f-1) + np.eye(nG)
#         next_prob = np.where(f == N, 1, 0)
#         return np.tile(next_states[:, np.newaxis, :], (1, nG, 1)), np.tile(next_prob, (nG, 1)) #just to cast into the appropriate dimension 
#     # if f[a] >= N: #if adding quantum to a tile where already has all quanta
#     #     return [f], [1.0] #just fixate at this state
#     # else:
#     nA = nG #for interpretability, set num of possible actions, which is equal to the number of tiles to allocate quanta
#     remove_quantum = np.tile(-np.eye(nG), (nA, 1, 1))
#     next_f = np.tile(f, (nA, nG, 1)) + remove_quantum  # Create a matrix with each row as allocations, dimension: nA x nS x nG
#     next_f[np.arange(nA),:,np.arange(nG)] += 1 # deterministically increase quanta for the acted tile.
#     np.fill_diagonal(next_f, next_f.diagonal() + 1) # ath row represents decrease quanta from unused pile
#     next_states = np.where(next_f < 0, 0, next_f) #nA by nG (nG number of possible future states (where to remove quantum)) by nG (array encoding allocation)

#     assigned_quanta = np.repeat(np.sum(f), nA) 
#     inv_disposible_quanta,unassigned_quanta = 1/(N - f), N - assigned_quanta
#     next_prob =  np.outer(inv_disposible_quanta.T, f) #nA by nG(nG number of possible future states)
#     np.fill_diagonal(next_prob, unassigned_quanta*inv_disposible_quanta)#probability of allocating from unassigned
        
#     return next_states, next_prob
#     # for g in range(nG): #enumerate possible tiles to decrease the quantum
#     #     if g != a: #definitely don't decrease the quantum from the selected tile
#     #         f_dec_assigned = f_next.copy()
#     #         f_dec_assigned[g] -= 1
#     #         p = f[g]/(N-f[a]) #probability proportional to the amount of quanta assigned to that tile
#     #         p_sum += p #increment to the total probability of decrease from assigned versus unassigned quanta
#     #         if p!=0:
#     #             possible_next.append((f_dec_assigned, p))
#     # possible_next.append((f_next, np.maximum(1-p_sum, 0.0))) #otherwise decrease on from unassigned quanta
#     # return possible_next

# def reward_assignment_solo(f, quanta_to_prob=quanta_to_prob_concave):
#     return np.mean(quanta_to_prob(beta, f, ns), axis = -1)
#     # return np.mean([reward_lookup[q] for q in f])

In [1]:
# %load_ext rpy2.ipython
# %matplotlib inline

import numpy as np
import pandas as pd
import json
import itertools as it
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

# %R library(lubridate)
# %R library(gganimate)
# %R library(tidyverse)

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns

# Simulate greedy algorithms

In [3]:
'''Task parameters'''
grid_size = 6 # assume 3 by 3 grid for now
G = np.array(list(it.product(range(grid_size), range(grid_size))))
nG = len(G)
ns = 4 # 4 distinct images
nP = 2 # 2 players
A = np.array(list(it.product(range(nG), repeat=nP))) #joint action space
nA = len(A)
horizon = 50 

'''Model parameters'''
N = nG # same quanta as n grids
beta = 1 
kappa = 0.01
nSim = 1

'''Manhattan distance'''
# manhattan_distance = lambda tuple1, tuple2:sum(abs(x - y) for x, y in zip(tuple1, tuple2))
# D = np.array([[np.sum([manhattan_distance(G[action_before[i]], G[action_now[i]]) for i in range(nP)]) for action_now in A] for action_before in A])
# D_cost = D/np.sum(D, axis=-1)
#is_possible is 1 if both players' next action is within 1 step
is_possible = np.array([[np.prod([np.max(abs(G[action_before[i]] - G[action_now[i]])) <= 1 for i in range(nP)]) for action_now in A] for action_before in A])

'''Different recall functions'''
quanta_to_prob_concave = lambda beta,q,ns: (2/ns-1) + (2-2/ns)/(1+np.exp(-beta*q)) #concave model
quanta_to_prob_inflect = lambda beta,q,ns: 1/(1+np.exp(np.log(ns-1)-beta*q)) #non-monotonic model
quanta_to_prob_convex = lambda beta,q,ns: (beta*q**2+ns)/(beta*N**2+ns) #convex model

def transistion_policy(f):
    '''
    This function takes in a state (particular quanta allocation), and output all possible next states and their probabilities
    f: (nP by nG) encodes quanta allocation of all players constituting the current state 
    next_states: (nA = nG^nP by nD = nG^nP by nP by nG) encodes all possible states to transition next. 
        The transition rule depends on the tuple of actions each player chooses. Each player can choose to maintain 1 tile out of nG tiles, 
        hence nA = nG^nP. Once a tile is maintained, one quantum is added to that tile for that player. However, there is no free lunch.
        The added quantum must come from either the existing quanta in one other tile, or from one of the unallocated quanta, if there is any.
        Therefore for each player, there are nG possible sources to move the quantum to add to the maintained tile 
        (nG-1 other tiles, and from the unallocated pile of quanta). To represent the possibility that the quantum comes from the unallocated pile of quanta,
        we just encode it using the same index as the mainatained tile. For example, if a player maintained tile 2, and the second index is 1 (or 3 or 5, etc), it means
        one quantum is allocated to tile 2 and tile 1 (or 3 or 5) looses one quantum. If a player maintained tile 2, and the second index is 2 as well,
        it means one quantum is allocated to tile 2 from the unallocated pile. 
        nP by nG matrix simply encodes the state that is transitioned to, as the consequence of the combination of maintaince actions and decay sources
        of all the players. 
    next_prob: (nA = nG^nP by nD = nG^nP) It is the same but instead of encoding allocation matrix, it encodes one number which is the transition possibility.
        the transition possibility is proportional to the amount of quanta at each decayed tile (an at the unallocated pile): P(g)=q_g/(N - q_a). 
        No quanta means 0 transition possibility

    There is only one exception scenario: if a tile already has N quanta, and yet the player still chooses to maintain that tile. Here we just stop increasing 
    the quanta at the maintained tile because N is the total amount of quanta. N remains N at that tile. The transition probability also is different in this case.
    For that player, no other tiles can decay quanta (because they alreay have no quantum), nor can the unallocated pile (all were allocated to the maintained tile). 
    Here the second index must mean something different than the unallocated pile. It just encodes probability of 1 (instead of 0), signifying that maintaining
    a tile with N quanta just stays at that state with probability 1. Maintaining other tiles does cause an exception because N would just become N-1 for sure
    to reallocate one quantum to the maintained tile.
    '''
    assert np.all(np.sum(f, axis=-1) <= N) #make sure allocation sum up to NO more than N for each player, because N is the max # of quanta
    nA = nG**nP #join action space
    nD = nG**nP #possible sources to decay quanta
    '''
    idx_mask:
    for each action tuple OR decay source (because they are the same number), 
    create nP by nG placeholder matrix. 1 if the index is what the tile the player acted on/decayed from, 0 otherwise.
    This is used to mask other variables for convenience. This solves the main coding challange that the first and second index
    are only integers that represent the tuples of player actions, but not tuples of tile index themselves.
    A: a list of all possible tuples of player actions (tile index) defined in the global environment. 
    '''
    idx_mask = np.take(np.eye(nG), A, axis=0) #nA by nP by nG
    add_quantum = np.tile(idx_mask[:, np.newaxis, :, :], (1,nD,1,1)) #repeat idx_mask nD times
    '''In add_quantum, it represents the places to add one quantum (tiles acted upon to maintain) '''
    remove_quantum = np.tile(idx_mask, (nA, 1,1,1)) #repeat idx_mask nA times
    '''In remove_quantum, it represents all possible places to decay one quantum (from other tiles or unallocated pile)'''
    f_tiled = np.tile(f, [nA, nD]+[1,1]) #recast current state into output dimension
    next_f = f_tiled - remove_quantum + add_quantum #add the quantum to maintained tile, and decay from other places
    add_again = add_quantum+remove_quantum == 2 # deterministically increase quanta for the acted tile
    '''add_again encodes locations that were the quantum added to the maintained tile come from the unallocated piles.'''
    '''Because unallocated piles is indexed by the same index as the maintained tile, remove_quantum does not know it and unecessarily decreased a quantum from the maintained tile.'''
    next_f += add_again #add that quantum back to the maintained tile.
    next_states = np.where(next_f < 0, 0, next_f) # if fall below 0 quantum, adjust back for you cannot have fewer than 0 quanta allocation.
    next_states = np.where(next_states > N, N, next_states) #if exceed N, does not increase anymore
    '''Now we finished all future possible states. Let us compute the transition possibilities'''
    unassigned_quanta,disposible_quanta  = N - np.sum(f, axis=-1), np.where(f == N, N, N - f) 
    '''
    assigned_quanta is length nP, encoding amount of unallocated quanta for each player
    disposible_quanta (nP by nG) assumes each play to act on each possible tiles, encodes N-q_a= total quanta outside of the maintained tile.
    If one tile already has N quanta, N-N=0 which cannot be divided. Just artificially make it N. These exceptions will be dealt later.
    '''
    inv_disposible_quanta = np.sum(idx_mask/disposible_quanta, axis=-1) #nD by nP: 1/(N-q_a) for each decayed g
    from_otherTiles =  f_tiled*remove_quantum*inv_disposible_quanta[:, np.newaxis, :, np.newaxis] 
    ''' 
    #f_tiled*remove_quantum masks to only the decayed tiles. *inv_disposible_quanta computes q_g * 1/(N-q_a)
    #now next_prob (nA by nD) has q_g * 1/(N-q_a), which is the probability quantum decays from each other tile.
    #the only entries needs replacements are entries where decay tile index equals the maintained tile index, for those probabilities are calculated differently.
    '''
    from_unassigned = unassigned_quanta*inv_disposible_quanta
    #from_unassigned(nA by nP): q_unassigned / N-q_a. q_unassigned=0 in case of exception (N quanta assigned to the maintained tile)
    stationary_state = add_again*(f==N)
    ''' 
    stationary_state takes care of the exception. If there is no exception (no tile has all N quanta), this is a zero matrix.
    If a tile of a play has N quanta, and that play still maintains that tile, then the corresponding entry is 1, representing p=1
    so the next state is the same as the current state where that player all-in on this tile.
    '''
    next_prob = add_again*from_unassigned[:, np.newaxis, :, np.newaxis]+ stationary_state + (1-add_again)*from_otherTiles 
    ''' 
    if decay index equals the maintained tile index, use probability value from unassigned pile. 
    At entries that has exception, unassigned p is 0, so add 1 from matrix stationary_state
    if decay index is not equal to the maintained tile index, just use the otherTiles
    '''
    next_prob = np.prod(np.sum(next_prob, axis=-1), axis=-1) #joint from all player's transitions
    return next_states,next_prob
            
def reward_assignment(f, quanta_to_prob=quanta_to_prob_concave):
    '''
    f: nPossibilities by nPlaye by nTiles
    '''
    p_noOne_recall = np.prod(1 - quanta_to_prob(beta, f, ns), axis=-2)
    p_someone_recall = 1-p_noOne_recall
    return np.mean(p_someone_recall, axis=-1)
    # for s in f:
    #     p_not_recall.append(1-np.mean([quanta_to_prob(beta,q,ns) for q in f]))
    # return 1-np.prod(p_not_recall)

f0 = np.zeros((nP,nG)) # initial assignments to be 0
def simulate(step_model, f0=f0,action_before="None",output = [], horizon = horizon, recall_function = quanta_to_prob_concave):
    if horizon % 10 ==0:
        print(horizon)
    # print(horizon)
    if horizon == 0:
        action_trajectory,allocation_trajectory = zip(*output)
        # Create a dictionary with the arrays
        sim_data = {
            "action_trajectory": [int(e) for e in np.array(action_trajectory)],
            "allocation_trajectory": np.array(allocation_trajectory).tolist()
        }
        return sim_data
    else:
        next_action, next_state = step_model(f0,action_before, recall_function=recall_function)
        return simulate(step_model=step_model, f0=next_state, action_before=next_action, output = output+[(next_action, next_state)], horizon = horizon-1, recall_function = recall_function)

## Baseline repulsion model

In [30]:
model_name = "repulsion_heuristics"
sigma = 11.5 #base standard deviation for directional noise
initial_actions = np.array([[3,4], [4,3]]) #initialize the first actions

def repulsion(initial_actions, horizon=horizon):
    previous_actions = initial_actions
    action_trajectory = []
    for t in range(horizon):
        # print(t)
        tile_indices = [np.where((G == np.floor(tile)).all(axis=1))[0][0] for tile in previous_actions]
        action_trajectory.append(int(np.where((A == tile_indices).all(axis=1))[0][0]))

        delta_c = - np.array([previous_actions[1]-previous_actions[0], previous_actions[0]-previous_actions[1]])
        delta_c_norm = np.linalg.norm(delta_c)
        within_bound = False
        while not within_bound:
            angle_err = np.array([np.radians(np.random.normal(0, sigma*delta_c_norm)), np.radians(np.random.normal(0, sigma*delta_c_norm))])
            cos_angles, sin_angles = np.cos(angle_err), np.sin(angle_err)
            # Stack the cosines and sines to form the rotation matrices
            rotation_matrices = np.stack([cos_angles, -sin_angles, sin_angles, cos_angles], axis=-1).reshape(-1, 2, 2)
            rotated_vectors = np.einsum('ijk,ik->ij', rotation_matrices, delta_c)
            new_c = rotated_vectors/10*np.linalg.norm(rotated_vectors, axis=1) + previous_actions
            if np.all((new_c >= 0) & (new_c <= grid_size)):
                within_bound = True
        previous_actions = new_c
        
    return action_trajectory

action_trajectory = repulsion(initial_actions)

actions = np.zeros((nSim, horizon),dtype=int)
actions[0,:] = action_trajectory

player1 = [a for s in range(nSim) for a in list(A[actions[s]][:,0]+1)]
player2 = [a for s in range(nSim) for a in list(A[actions[s]][:,1]+1)]
dyad_currentRound = [r+1 for s in range(nSim) for r in [s]*horizon]
df_sim = pd.DataFrame({'idx_tile_hovered':player1+player2, 
                       'playerID':[1]*horizon*nSim + [2]*horizon*nSim, 
                       'nFrame': list(range(horizon))*nSim+list(range(horizon))*nSim,
                       'dyad_currentRound': dyad_currentRound*2})

# %R -i df_sim -i grid_size -i model_name

In [31]:
# Create layout DataFrame
layout = pd.DataFrame({
    'x': np.tile(np.arange(1, grid_size + 1) - 0.5, grid_size),
    'y': np.repeat(np.arange(grid_size, 0, -1) - 0.5, grid_size),
    'idx': np.arange(1, grid_size**2 + 1)
})

df_animate = df_sim.groupby('playerID').apply(lambda x: x.assign(frame_order=np.arange(1, len(x) + 1))).reset_index(drop=True)
#df_animate = df_animate.assign(idx=lambda x: x.apply(lambda row: np.arange(1, grid_size**2 + 1), axis=1))
#df_animate = df_animate.explode('idx')
#df_animate['is_hovered'] = df_animate['idx'] == df_animate['idx_tile_hovered']
#df_animate['visited_state'] = df_animate.groupby(['idx', 'playerID'])['is_hovered'].cumsum()

pd.set_option('display.max_rows', None)
display(df_animate.loc[df_animate.frame_order < 10])

,idx_tile_hovered,playerID,nFrame,dyad_currentRound,frame_order
0,23,1,0,1,1
1,17,1,1,1,2
2,17,1,2,1,3
3,11,1,3,1,4
4,10,1,4,1,5
5,4,1,5,1,6
6,22,1,6,1,7
7,22,1,7,1,8
8,22,1,8,1,9
50,28,2,0,1,1


In [23]:
df_animate

,idx_tile_hovered,playerID,nFrame,dyad_currentRound,frame_order,idx,is_hovered,visited_state
0,23,1,0,1,1,1,False,0
0,23,1,0,1,1,2,False,0
0,23,1,0,1,1,3,False,0
0,23,1,0,1,1,4,False,0
0,23,1,0,1,1,5,False,0
...,...,...,...,...,...,...,...,...
99,7,2,49,1,50,32,False,2
99,7,2,49,1,50,33,False,6
99,7,2,49,1,50,34,False,3
99,7,2,49,1,50,35,False,2


In [73]:
%%R
layout <- data.frame(
  x = rep(1:grid_size, grid_size)-0.5,
  y = rep(grid_size:1, each = grid_size)-0.5,
  idx = 1:grid_size^2
)

df_animate = df_sim%>% #set the resolution to the shortest duration of viewing a tile.
group_by(playerID)%>% 
mutate(frame_order = row_number())%>% ungroup()%>%
  mutate(idx = list(seq_len(grid_size^2))) %>%
  unnest(idx) %>%
  mutate(is_hovered = idx == idx_tile_hovered)%>%
  group_by(idx, playerID) %>%
  mutate(visited_state = cumsum(is_hovered)) %>%ungroup()%>%
  uncount(nFrame)

data_merged <-  merge(layout, df_animate, by = "idx", all.x = TRUE)%>%
arrange(frame_order)%>%group_by(idx, playerID)%>%
mutate(row_id = ifelse(!is.na(frame_order), row_number(), NaN))%>%ungroup()%>%
  mutate(combined_column = paste0(row_id, " - ", dyad_currentRound)) 

# Create a base plot
plot <- ggplot(data_merged, aes(x = x, y = y)) +
geom_tile(aes(fill = ifelse(is_hovered, NA, visited_state)), size = 0.5) +
  scale_fill_gradient(
    low = "white",
    high = "blue",
    na.value = "black",
    guide = FALSE
  )+
  labs(x = "X", y = "Y", fill = "Color") +
  coord_equal() +
  theme_minimal() +
  scale_x_continuous(expand = c(0, 0)) +
  scale_y_continuous(expand = c(0, 0)) +
  labs(title = "Tile Viewed Over Time") + facet_wrap(~ playerID)

animated_plot <- plot+
  transition_states(
    row_id,
    transition_length = 0,
    state_length = 1
) +
  ease_aes('cubic-in-out') +
  enter_fade() +
  exit_fade()+
  ggtitle("Time: {closest_state}")

# View the animated plot
animate(animated_plot, nframes = 100, fps = 10, width = 300, height = 300)
anim_save(str_c(model_name, " ", ".gif") , animation = last_animation())


Inserting image 1 at 0.00s (1%)...
Inserting image 2 at 0.10s (2%)...
Inserting image 3 at 0.20s (3%)...
Inserting image 4 at 0.30s (4%)...
Inserting image 5 at 0.40s (5%)...
Inserting image 6 at 0.50s (6%)...
Inserting image 7 at 0.60s (7%)...
Inserting image 8 at 0.70s (8%)...
Inserting image 9 at 0.80s (9%)...
Inserting image 10 at 0.90s (10%)...
Inserting image 11 at 1.00s (11%)...
Inserting image 12 at 1.10s (12%)...
Inserting image 13 at 1.20s (13%)...
Inserting image 14 at 1.30s (14%)...
Inserting image 15 at 1.40s (15%)...
Inserting image 16 at 1.50s (16%)...
Inserting image 17 at 1.60s (17%)...
Inserting image 18 at 1.70s (18%)...
Inserting image 19 at 1.80s (19%)...
Inserting image 20 at 1.90s (20%)...
Inserting image 21 at 2.00s (21%)...
Inserting image 22 at 2.10s (22%)...
Inserting image 23 at 2.20s (23%)...
Inserting image 24 at 2.30s (24%)...
Inserting image 25 at 2.40s (25%)...
Inserting image 26 at 2.50s (26%)...
Inserting image 27 at 2.60s (27%)...
Inserting image 28

## Mode 1: Dyad_central_greedy

In this model, agents can only select neighboring tiles as next action. They choose so that they can have the best next allocation. It is a central planner model so both agents act according to the central best plan.

In [34]:
model_name = "Dyad_central_greedy_concave"

def simulate_step_centralPlanner(current_f, action_before="None", recall_function = quanta_to_prob_concave):
    next_states, next_prob = transistion_policy(current_f) 
    if action_before != "None": 
        available_actions = np.where(is_possible[action_before]==1)[0]
        # find action index where both players move maximum 1 step away from current tiles
        next_states, next_prob = next_states[available_actions], next_prob[available_actions]
        # find all possible next states consistent with available actions
    else:
        available_actions = np.arange(next_states.shape[0])#first action has no spatial contraint
    # if action_before == "None":
    #     distance_costs = 0
    # else:
    #     distance_costs = D_cost[action_before]    

    rewards = reward_assignment(next_states, quanta_to_prob=recall_function)
    exp_reward = np.log(np.sum(rewards*next_prob, axis=-1)) #- distance_costs
    best_actions = np.where(exp_reward == np.max(exp_reward))[0] #find index of available actions that maximize expected reward
    next_action = np.random.choice(best_actions) #choose the index
    next_f, next_p = next_states[next_action], next_prob[next_action]
    next_state_idx = np.random.choice(next_f.shape[0], p=next_p/next_p.sum()) #sample the next state
    return available_actions[next_action], next_f[next_state_idx]

for i in range(nSim):
    if i % 10 ==0:
        print(i)
    sim_data = simulate(simulate_step_centralPlanner)
    # Save the dictionary to a JSON file
    with open(str(i)+"_"+model_name +".json", "w") as file:
        json.dump(sim_data, file)

actions, rewards = np.zeros((nSim, horizon),dtype=int), np.zeros((nSim, horizon))
for i in range(nSim):
    with open(str(i)+"_"+model_name +".json") as json_file:
        # Load the JSON data into a Python dictionary
        data = json.load(json_file)
        actions[i,:] = data["action_trajectory"]
        rewards[i,:] = [reward_assignment(np.array(state)) for state in data["allocation_trajectory"]]

player1 = [a for s in range(nSim) for a in list(A[actions[s]][:,0]+1)]
player2 = [a for s in range(nSim) for a in list(A[actions[s]][:,1]+1)]
dyad_currentRound = [r+1 for s in range(nSim) for r in [s]*horizon]
df_sim = pd.DataFrame({'idx_tile_hovered':player1+player2, 
                       'playerID':[1]*horizon*nSim + [2]*horizon*nSim, 
                       'nFrame': list(range(horizon))*nSim+list(range(horizon))*nSim,
                       'dyad_currentRound': dyad_currentRound*2})

# %R -i df_sim -i grid_size -i model_name

0
50
40
30
20
10
0


In [66]:
%%R
layout <- data.frame(
  x = rep(1:grid_size, grid_size)-0.5,
  y = rep(grid_size:1, each = grid_size)-0.5,
  idx = 1:grid_size^2
)

df_animate = df_sim%>% #set the resolution to the shortest duration of viewing a tile.
group_by(playerID)%>% 
mutate(frame_order = row_number())%>% ungroup()%>%
  mutate(idx = list(seq_len(grid_size^2))) %>%
  unnest(idx) %>%
  mutate(is_hovered = idx == idx_tile_hovered)%>%
  group_by(idx, playerID) %>%
  mutate(visited_state = cumsum(is_hovered)) %>%ungroup()%>%
  uncount(nFrame)

data_merged <-  merge(layout, df_animate, by = "idx", all.x = TRUE)%>%
arrange(frame_order)%>%group_by(idx, playerID)%>%
mutate(row_id = ifelse(!is.na(frame_order), row_number(), NaN))%>%ungroup()%>%
  mutate(combined_column = paste0(row_id, " - ", dyad_currentRound)) 

# Create a base plot
plot <- ggplot(data_merged, aes(x = x, y = y)) +
geom_tile(aes(fill = ifelse(is_hovered, NA, visited_state)), size = 0.5) +
  scale_fill_gradient(
    low = "white",
    high = "blue",
    na.value = "black",
    guide = FALSE
  )+
  labs(x = "X", y = "Y", fill = "Color") +
  coord_equal() +
  theme_minimal() +
  scale_x_continuous(expand = c(0, 0)) +
  scale_y_continuous(expand = c(0, 0)) +
  labs(title = "Tile Viewed Over Time") + facet_wrap(~ playerID)

animated_plot <- plot+
  transition_states(
    row_id,
    transition_length = 0,
    state_length = 1
) +
  ease_aes('cubic-in-out') +
  enter_fade() +
  exit_fade()+
  ggtitle("Time: {closest_state}")

# View the animated plot
animate(animated_plot, nframes = 100, fps = 10, width = 300, height = 300)
anim_save(str_c(model_name, " ", ".gif") , animation = last_animation())


Inserting image 1 at 0.00s (1%)...
Inserting image 2 at 0.10s (2%)...
Inserting image 3 at 0.20s (3%)...
Inserting image 4 at 0.30s (4%)...
Inserting image 5 at 0.40s (5%)...
Inserting image 6 at 0.50s (6%)...
Inserting image 7 at 0.60s (7%)...
Inserting image 8 at 0.70s (8%)...
Inserting image 9 at 0.80s (9%)...
Inserting image 10 at 0.90s (10%)...
Inserting image 11 at 1.00s (11%)...
Inserting image 12 at 1.10s (12%)...
Inserting image 13 at 1.20s (13%)...
Inserting image 14 at 1.30s (14%)...
Inserting image 15 at 1.40s (15%)...
Inserting image 16 at 1.50s (16%)...
Inserting image 17 at 1.60s (17%)...
Inserting image 18 at 1.70s (18%)...
Inserting image 19 at 1.80s (19%)...
Inserting image 20 at 1.90s (20%)...
Inserting image 21 at 2.00s (21%)...
Inserting image 22 at 2.10s (22%)...
Inserting image 23 at 2.20s (23%)...
Inserting image 24 at 2.30s (24%)...
Inserting image 25 at 2.40s (25%)...
Inserting image 26 at 2.50s (26%)...
Inserting image 27 at 2.60s (27%)...
Inserting image 28

## Model 2: Iterative simulation

in this model, each player assumes the other player will take the same action as previous timepoint. Then greedy find the best action conditional on this information. Spatial constraint still applies.

In [35]:
model_name = "Dyad_iterative_greedy"

initial_actions, f1 = simulate_step_centralPlanner(f0)
def find_possible_actions(matrix, action_before, player):
    # Create a boolean mask for the columns other than the i-th column
    mask = np.ones(matrix.shape[1], dtype=bool)
    mask[player] = False

    # Compare the matrix rows with the array using the mask
    matching_rows = np.all(matrix[:, mask] == action_before[mask], axis=1)

    # Get the row indices where the matching condition is true
    row_indices = np.where(matching_rows)[0]

    return row_indices
def simulate_step_iterative(current_f, action_before=initial_actions, recall_function = quanta_to_prob_concave):
    next_states, next_prob = transistion_policy(current_f) 
    available_actions = np.where(is_possible[action_before]==1)[0]

    #distance_costs = D_cost[action_before]    
    next_action = np.zeros(nP, dtype=int)
    for player in range(nP):
        allowed_row_idx = np.intersect1d(find_possible_actions(A, A[action_before],player),available_actions)
        #find allowed next actions but also conditional on partner picking the same as previous actions.
        allowed_states, allowed_probs = next_states[allowed_row_idx],next_prob[allowed_row_idx]
        #filter future states compatible with partner's action
        rewards = reward_assignment(allowed_states, quanta_to_prob=recall_function)
        exp_reward = np.log(np.sum(rewards*allowed_probs, axis=-1)) #- distance_costs[allowed_row_idx]
        best_actions = allowed_row_idx[np.where(exp_reward == np.max(exp_reward))[0]]
        next_action[player] = A[np.random.choice(best_actions)][player]

    next_action_index = np.where((A == next_action).all(axis=1))[0]
    assert len(next_action_index) == 1
    next_f, next_p = next_states[next_action_index[0]], next_prob[next_action_index[0]]
    next_state_idx = np.random.choice(next_f.shape[0], p=next_p/next_p.sum())
    return next_action_index[0], next_f[next_state_idx]

for i in range(nSim):
    sim_data = simulate(simulate_step_iterative,f0=f1,action_before=initial_actions)
    # Save the dictionary to a JSON file
    with open(str(i)+"_"+model_name +".json", "w") as file:
        json.dump(sim_data, file)

actions, rewards = np.zeros((nSim, horizon),dtype=int), np.zeros((nSim, horizon))
for i in range(nSim):
    with open(str(i)+"_"+model_name +".json") as json_file:
        # Load the JSON data into a Python dictionary
        data = json.load(json_file)
        actions[i,:] = data["action_trajectory"]
        rewards[i,:] = [reward_assignment(np.array(state)) for state in data["allocation_trajectory"]]

player1 = [a for s in range(nSim) for a in list(A[actions[s]][:,0]+1)]
player2 = [a for s in range(nSim) for a in list(A[actions[s]][:,1]+1)]
dyad_currentRound = [r+1 for s in range(nSim) for r in [s]*horizon]
df_sim = pd.DataFrame({'idx_tile_hovered':player1+player2, 
                       'playerID':[1]*horizon*nSim + [2]*horizon*nSim, 
                       'nFrame': list(range(horizon))*nSim+list(range(horizon))*nSim,
                       'dyad_currentRound': dyad_currentRound*2})

# %R -i df_sim -i grid_size -i model_name

50
40
30
20
10
0


In [68]:
%%R
layout <- data.frame(
  x = rep(1:grid_size, grid_size)-0.5,
  y = rep(grid_size:1, each = grid_size)-0.5,
  idx = 1:grid_size^2
)

df_animate = df_sim%>% #set the resolution to the shortest duration of viewing a tile.
group_by(playerID)%>% 
mutate(frame_order = row_number())%>% ungroup()%>%
  mutate(idx = list(seq_len(grid_size^2))) %>%
  unnest(idx) %>%
  mutate(is_hovered = idx == idx_tile_hovered)%>%
  group_by(idx, playerID) %>%
  mutate(visited_state = cumsum(is_hovered)) %>%ungroup()%>%
  uncount(nFrame)

data_merged <-  merge(layout, df_animate, by = "idx", all.x = TRUE)%>%
arrange(frame_order)%>%group_by(idx, playerID)%>%
mutate(row_id = ifelse(!is.na(frame_order), row_number(), NaN))%>%ungroup()%>%
  mutate(combined_column = paste0(row_id, " - ", dyad_currentRound)) 

# Create a base plot
plot <- ggplot(data_merged, aes(x = x, y = y)) +
geom_tile(aes(fill = ifelse(is_hovered, NA, visited_state)), size = 0.5) +
  scale_fill_gradient(
    low = "white",
    high = "blue",
    na.value = "black",
    guide = FALSE
  )+
  labs(x = "X", y = "Y", fill = "Color") +
  coord_equal() +
  theme_minimal() +
  scale_x_continuous(expand = c(0, 0)) +
  scale_y_continuous(expand = c(0, 0)) +
  labs(title = "Tile Viewed Over Time") + facet_wrap(~ playerID)

animated_plot <- plot+
  transition_states(
    row_id,
    transition_length = 0,
    state_length = 1
) +
  ease_aes('cubic-in-out') +
  enter_fade() +
  exit_fade()+
  ggtitle("Time: {closest_state}")

# View the animated plot
animate(animated_plot, nframes = 100, fps = 10, width = 300, height = 300)
anim_save(str_c(model_name, " ", ".gif") , animation = last_animation())


Inserting image 1 at 0.00s (1%)...
Inserting image 2 at 0.10s (2%)...
Inserting image 3 at 0.20s (3%)...
Inserting image 4 at 0.30s (4%)...
Inserting image 5 at 0.40s (5%)...
Inserting image 6 at 0.50s (6%)...
Inserting image 7 at 0.60s (7%)...
Inserting image 8 at 0.70s (8%)...
Inserting image 9 at 0.80s (9%)...
Inserting image 10 at 0.90s (10%)...
Inserting image 11 at 1.00s (11%)...
Inserting image 12 at 1.10s (12%)...
Inserting image 13 at 1.20s (13%)...
Inserting image 14 at 1.30s (14%)...
Inserting image 15 at 1.40s (15%)...
Inserting image 16 at 1.50s (16%)...
Inserting image 17 at 1.60s (17%)...
Inserting image 18 at 1.70s (18%)...
Inserting image 19 at 1.80s (19%)...
Inserting image 20 at 1.90s (20%)...
Inserting image 21 at 2.00s (21%)...
Inserting image 22 at 2.10s (22%)...
Inserting image 23 at 2.20s (23%)...
Inserting image 24 at 2.30s (24%)...
Inserting image 25 at 2.40s (25%)...
Inserting image 26 at 2.50s (26%)...
Inserting image 27 at 2.60s (27%)...
Inserting image 28

## Model 3: Same as model 1, but use inflective recall function

In [36]:
model_name = "Dyad_central_greedy_inflect"

for i in range(nSim):
    if i % 10 ==0:
        print(i)
    sim_data = simulate(simulate_step_centralPlanner, recall_function = quanta_to_prob_inflect)
    # Save the dictionary to a JSON file
    with open(str(i)+"_"+model_name +".json", "w") as file:
        json.dump(sim_data, file)

actions, rewards = np.zeros((nSim, horizon),dtype=int), np.zeros((nSim, horizon))
for i in range(nSim):
    with open(str(i)+"_"+model_name +".json") as json_file:
        # Load the JSON data into a Python dictionary
        data = json.load(json_file)
        actions[i,:] = data["action_trajectory"]
        rewards[i,:] = [reward_assignment(np.array(state)) for state in data["allocation_trajectory"]]

player1 = [a for s in range(nSim) for a in list(A[actions[s]][:,0]+1)]
player2 = [a for s in range(nSim) for a in list(A[actions[s]][:,1]+1)]
dyad_currentRound = [r+1 for s in range(nSim) for r in [s]*horizon]
df_sim = pd.DataFrame({'idx_tile_hovered':player1+player2, 
                       'playerID':[1]*horizon*nSim + [2]*horizon*nSim, 
                       'nFrame': list(range(horizon))*nSim+list(range(horizon))*nSim,
                       'dyad_currentRound': dyad_currentRound*2})

# %R -i df_sim -i grid_size -i model_name

0
50
40
30
20
10
0


In [70]:
%%R
layout <- data.frame(
  x = rep(1:grid_size, grid_size)-0.5,
  y = rep(grid_size:1, each = grid_size)-0.5,
  idx = 1:grid_size^2
)

df_animate = df_sim%>% #set the resolution to the shortest duration of viewing a tile.
group_by(playerID)%>% 
mutate(frame_order = row_number())%>% ungroup()%>%
  mutate(idx = list(seq_len(grid_size^2))) %>%
  unnest(idx) %>%
  mutate(is_hovered = idx == idx_tile_hovered)%>%
  group_by(idx, playerID) %>%
  mutate(visited_state = cumsum(is_hovered)) %>%ungroup()%>%
  uncount(nFrame)

data_merged <-  merge(layout, df_animate, by = "idx", all.x = TRUE)%>%
arrange(frame_order)%>%group_by(idx, playerID)%>%
mutate(row_id = ifelse(!is.na(frame_order), row_number(), NaN))%>%ungroup()%>%
  mutate(combined_column = paste0(row_id, " - ", dyad_currentRound)) 

# Create a base plot
plot <- ggplot(data_merged, aes(x = x, y = y)) +
geom_tile(aes(fill = ifelse(is_hovered, NA, visited_state)), size = 0.5) +
  scale_fill_gradient(
    low = "white",
    high = "blue",
    na.value = "black",
    guide = FALSE
  )+
  labs(x = "X", y = "Y", fill = "Color") +
  coord_equal() +
  theme_minimal() +
  scale_x_continuous(expand = c(0, 0)) +
  scale_y_continuous(expand = c(0, 0)) +
  labs(title = "Tile Viewed Over Time") + facet_wrap(~ playerID)

animated_plot <- plot+
  transition_states(
    row_id,
    transition_length = 0,
    state_length = 1
) +
  ease_aes('cubic-in-out') +
  enter_fade() +
  exit_fade()+
  ggtitle("Time: {closest_state}")

# View the animated plot
animate(animated_plot, nframes = 100, fps = 10, width = 300, height = 300)
anim_save(str_c(model_name, " ", ".gif") , animation = last_animation())


Inserting image 1 at 0.00s (1%)...
Inserting image 2 at 0.10s (2%)...
Inserting image 3 at 0.20s (3%)...
Inserting image 4 at 0.30s (4%)...
Inserting image 5 at 0.40s (5%)...
Inserting image 6 at 0.50s (6%)...
Inserting image 7 at 0.60s (7%)...
Inserting image 8 at 0.70s (8%)...
Inserting image 9 at 0.80s (9%)...
Inserting image 10 at 0.90s (10%)...
Inserting image 11 at 1.00s (11%)...
Inserting image 12 at 1.10s (12%)...
Inserting image 13 at 1.20s (13%)...
Inserting image 14 at 1.30s (14%)...
Inserting image 15 at 1.40s (15%)...
Inserting image 16 at 1.50s (16%)...
Inserting image 17 at 1.60s (17%)...
Inserting image 18 at 1.70s (18%)...
Inserting image 19 at 1.80s (19%)...
Inserting image 20 at 1.90s (20%)...
Inserting image 21 at 2.00s (21%)...
Inserting image 22 at 2.10s (22%)...
Inserting image 23 at 2.20s (23%)...
Inserting image 24 at 2.30s (24%)...
Inserting image 25 at 2.40s (25%)...
Inserting image 26 at 2.50s (26%)...
Inserting image 27 at 2.60s (27%)...
Inserting image 28

## Model 4: Same as model 1, but use convex recall function

In [71]:
model_name = "Dyad_central_greedy_convex"

for i in range(nSim):
    if i % 10 ==0:
        print(i)
    sim_data = simulate(simulate_step_centralPlanner, recall_function = quanta_to_prob_convex)
    # Save the dictionary to a JSON file
    with open(str(i)+"_"+model_name +".json", "w") as file:
        json.dump(sim_data, file)

actions, rewards = np.zeros((nSim, horizon),dtype=int), np.zeros((nSim, horizon))
for i in range(nSim):
    with open(str(i)+"_"+model_name +".json") as json_file:
        # Load the JSON data into a Python dictionary
        data = json.load(json_file)
        actions[i,:] = data["action_trajectory"]
        rewards[i,:] = [reward_assignment(np.array(state)) for state in data["allocation_trajectory"]]

player1 = [a for s in range(nSim) for a in list(A[actions[s]][:,0]+1)]
player2 = [a for s in range(nSim) for a in list(A[actions[s]][:,1]+1)]
dyad_currentRound = [r+1 for s in range(nSim) for r in [s]*horizon]
df_sim = pd.DataFrame({'idx_tile_hovered':player1+player2, 
                       'playerID':[1]*horizon*nSim + [2]*horizon*nSim, 
                       'nFrame': list(range(horizon))*nSim+list(range(horizon))*nSim,
                       'dyad_currentRound': dyad_currentRound*2})

%R -i df_sim -i grid_size -i model_name

0
50
40
30
20
10
0


In [72]:
%%R
layout <- data.frame(
  x = rep(1:grid_size, grid_size)-0.5,
  y = rep(grid_size:1, each = grid_size)-0.5,
  idx = 1:grid_size^2
)

df_animate = df_sim%>% #set the resolution to the shortest duration of viewing a tile.
group_by(playerID)%>% 
mutate(frame_order = row_number())%>% ungroup()%>%
  mutate(idx = list(seq_len(grid_size^2))) %>%
  unnest(idx) %>%
  mutate(is_hovered = idx == idx_tile_hovered)%>%
  group_by(idx, playerID) %>%
  mutate(visited_state = cumsum(is_hovered)) %>%ungroup()%>%
  uncount(nFrame)

data_merged <-  merge(layout, df_animate, by = "idx", all.x = TRUE)%>%
arrange(frame_order)%>%group_by(idx, playerID)%>%
mutate(row_id = ifelse(!is.na(frame_order), row_number(), NaN))%>%ungroup()%>%
  mutate(combined_column = paste0(row_id, " - ", dyad_currentRound)) 

# Create a base plot
plot <- ggplot(data_merged, aes(x = x, y = y)) +
geom_tile(aes(fill = ifelse(is_hovered, NA, visited_state)), size = 0.5) +
  scale_fill_gradient(
    low = "white",
    high = "blue",
    na.value = "black",
    guide = FALSE
  )+
  labs(x = "X", y = "Y", fill = "Color") +
  coord_equal() +
  theme_minimal() +
  scale_x_continuous(expand = c(0, 0)) +
  scale_y_continuous(expand = c(0, 0)) +
  labs(title = "Tile Viewed Over Time") + facet_wrap(~ playerID)

animated_plot <- plot+
  transition_states(
    row_id,
    transition_length = 0,
    state_length = 1
) +
  ease_aes('cubic-in-out') +
  enter_fade() +
  exit_fade()+
  ggtitle("Time: {closest_state}")

# View the animated plot
animate(animated_plot, nframes = 100, fps = 10, width = 300, height = 300)
anim_save(str_c(model_name, " ", ".gif") , animation = last_animation())


Inserting image 1 at 0.00s (1%)...
Inserting image 2 at 0.10s (2%)...
Inserting image 3 at 0.20s (3%)...
Inserting image 4 at 0.30s (4%)...
Inserting image 5 at 0.40s (5%)...
Inserting image 6 at 0.50s (6%)...
Inserting image 7 at 0.60s (7%)...
Inserting image 8 at 0.70s (8%)...
Inserting image 9 at 0.80s (9%)...
Inserting image 10 at 0.90s (10%)...
Inserting image 11 at 1.00s (11%)...
Inserting image 12 at 1.10s (12%)...
Inserting image 13 at 1.20s (13%)...
Inserting image 14 at 1.30s (14%)...
Inserting image 15 at 1.40s (15%)...
Inserting image 16 at 1.50s (16%)...
Inserting image 17 at 1.60s (17%)...
Inserting image 18 at 1.70s (18%)...
Inserting image 19 at 1.80s (19%)...
Inserting image 20 at 1.90s (20%)...
Inserting image 21 at 2.00s (21%)...
Inserting image 22 at 2.10s (22%)...
Inserting image 23 at 2.20s (23%)...
Inserting image 24 at 2.30s (24%)...
Inserting image 25 at 2.40s (25%)...
Inserting image 26 at 2.50s (26%)...
Inserting image 27 at 2.60s (27%)...
Inserting image 28

## Model 5: same as model 1, but reward function different

In [80]:
model_name = "Dyad_central_greedy_concave_unshared"

def reward_assignment(f, quanta_to_prob=quanta_to_prob_concave):
    '''
    f: nPossibilities by nPlaye by nTiles
    '''
    p_wrong = 1 - np.mean(quanta_to_prob(beta, f, ns), axis=-1)
    p_someone_right = 1-np.prod(p_wrong)
    return p_someone_right
    # for s in f:
    #     p_not_recall.append(1-np.mean([quanta_to_prob(beta,q,ns) for q in f]))
    # return 1-np.prod(p_not_recall)

def simulate_step_centralPlanner(current_f, action_before="None", recall_function = quanta_to_prob_concave):
    next_states, next_prob = transistion_policy(current_f) 
    if action_before != "None": 
        available_actions = np.where(is_possible[action_before]==1)[0]
        # find action index where both players move maximum 1 step away from current tiles
        next_states, next_prob = next_states[available_actions], next_prob[available_actions]
        # find all possible next states consistent with available actions
    else:
        available_actions = np.arange(next_states.shape[0])#first action has no spatial contraint
    # if action_before == "None":
    #     distance_costs = 0
    # else:
    #     distance_costs = D_cost[action_before]    

    rewards = reward_assignment(next_states, quanta_to_prob=recall_function)
    exp_reward = np.log(np.sum(rewards*next_prob, axis=-1)) #- distance_costs
    best_actions = np.where(exp_reward == np.max(exp_reward))[0] #find index of available actions that maximize expected reward
    next_action = np.random.choice(best_actions) #choose the index
    next_f, next_p = next_states[next_action], next_prob[next_action]
    next_state_idx = np.random.choice(next_f.shape[0], p=next_p/next_p.sum()) #sample the next state
    return available_actions[next_action], next_f[next_state_idx]

for i in range(nSim):
    if i % 10 ==0:
        print(i)
    sim_data = simulate(simulate_step_centralPlanner)
    # Save the dictionary to a JSON file
    with open(str(i)+"_"+model_name +".json", "w") as file:
        json.dump(sim_data, file)

actions, rewards = np.zeros((nSim, horizon),dtype=int), np.zeros((nSim, horizon))
for i in range(nSim):
    with open(str(i)+"_"+model_name +".json") as json_file:
        # Load the JSON data into a Python dictionary
        data = json.load(json_file)
        actions[i,:] = data["action_trajectory"]
        rewards[i,:] = [reward_assignment(np.array(state)) for state in data["allocation_trajectory"]]

player1 = [a for s in range(nSim) for a in list(A[actions[s]][:,0]+1)]
player2 = [a for s in range(nSim) for a in list(A[actions[s]][:,1]+1)]
dyad_currentRound = [r+1 for s in range(nSim) for r in [s]*horizon]
df_sim = pd.DataFrame({'idx_tile_hovered':player1+player2, 
                       'playerID':[1]*horizon*nSim + [2]*horizon*nSim, 
                       'nFrame': list(range(horizon))*nSim+list(range(horizon))*nSim,
                       'dyad_currentRound': dyad_currentRound*2})

%R -i df_sim -i grid_size -i model_name

0
50
40
30
20
10
0


In [81]:
%%R
layout <- data.frame(
  x = rep(1:grid_size, grid_size)-0.5,
  y = rep(grid_size:1, each = grid_size)-0.5,
  idx = 1:grid_size^2
)

df_animate = df_sim%>% #set the resolution to the shortest duration of viewing a tile.
group_by(playerID)%>% 
mutate(frame_order = row_number())%>% ungroup()%>%
  mutate(idx = list(seq_len(grid_size^2))) %>%
  unnest(idx) %>%
  mutate(is_hovered = idx == idx_tile_hovered)%>%
  group_by(idx, playerID) %>%
  mutate(visited_state = cumsum(is_hovered)) %>%ungroup()%>%
  uncount(nFrame)

data_merged <-  merge(layout, df_animate, by = "idx", all.x = TRUE)%>%
arrange(frame_order)%>%group_by(idx, playerID)%>%
mutate(row_id = ifelse(!is.na(frame_order), row_number(), NaN))%>%ungroup()%>%
  mutate(combined_column = paste0(row_id, " - ", dyad_currentRound)) 

# Create a base plot
plot <- ggplot(data_merged, aes(x = x, y = y)) +
geom_tile(aes(fill = ifelse(is_hovered, NA, visited_state)), size = 0.5) +
  scale_fill_gradient(
    low = "white",
    high = "blue",
    na.value = "black",
    guide = FALSE
  )+
  labs(x = "X", y = "Y", fill = "Color") +
  coord_equal() +
  theme_minimal() +
  scale_x_continuous(expand = c(0, 0)) +
  scale_y_continuous(expand = c(0, 0)) +
  labs(title = "Tile Viewed Over Time") + facet_wrap(~ playerID)

animated_plot <- plot+
  transition_states(
    row_id,
    transition_length = 0,
    state_length = 1
) +
  ease_aes('cubic-in-out') +
  enter_fade() +
  exit_fade()+
  ggtitle("Time: {closest_state}")

# View the animated plot
animate(animated_plot, nframes = 100, fps = 10, width = 300, height = 300)
anim_save(str_c(model_name, " ", ".gif") , animation = last_animation())


Inserting image 1 at 0.00s (1%)...
Inserting image 2 at 0.10s (2%)...
Inserting image 3 at 0.20s (3%)...
Inserting image 4 at 0.30s (4%)...
Inserting image 5 at 0.40s (5%)...
Inserting image 6 at 0.50s (6%)...
Inserting image 7 at 0.60s (7%)...
Inserting image 8 at 0.70s (8%)...
Inserting image 9 at 0.80s (9%)...
Inserting image 10 at 0.90s (10%)...
Inserting image 11 at 1.00s (11%)...
Inserting image 12 at 1.10s (12%)...
Inserting image 13 at 1.20s (13%)...
Inserting image 14 at 1.30s (14%)...
Inserting image 15 at 1.40s (15%)...
Inserting image 16 at 1.50s (16%)...
Inserting image 17 at 1.60s (17%)...
Inserting image 18 at 1.70s (18%)...
Inserting image 19 at 1.80s (19%)...
Inserting image 20 at 1.90s (20%)...
Inserting image 21 at 2.00s (21%)...
Inserting image 22 at 2.10s (22%)...
Inserting image 23 at 2.20s (23%)...
Inserting image 24 at 2.30s (24%)...
Inserting image 25 at 2.40s (25%)...
Inserting image 26 at 2.50s (26%)...
Inserting image 27 at 2.60s (27%)...
Inserting image 28

# Analysis of rewardingness of quanta assignment

In [16]:
grid_size = 3 # assume 3 by 3 grid for now
G = list(it.product(range(grid_size), range(grid_size)))
ns = 4 # 4 distinct images
nG = len(G)
D = np.array([[manhattan_distance(tuple1, tuple2) for tuple2 in G] for tuple1 in G])
T = 40 
N = 4*nG
beta = 1
kappa = 0.01

def create_random_array(k=nG ,N=N):
    # Generate a random array of length 36 with values between 0 and 1
    arr = np.random.random(k)

    # Scale the array to have a sum less than N
    arr = arr * (N / arr.sum())

    # Convert the array to integers
    arr = arr.astype(int)

    return arr      

def compute_shannon_entropy(unnormalized_array):
    # Normalize the array to get a probability distribution
    total = np.sum(unnormalized_array)
    probabilities = unnormalized_array / total
    
    # Compute Shannon entropy
    entropy = -np.sum(probabilities * np.log2(probabilities))
    
    return entropy

def terminal_states_solo(N=N,k=nG):
    # This is a helper function to perform the recursive generation
    def helper(current_array, current_sum):
        if len(current_array) == k:
            if current_sum == N:
                result.append(np.array(current_array.copy()))
            return
        for i in range(0, N - current_sum - (k - len(current_array) - 1) + 1):
            current_array.append(i)
            helper(current_array, current_sum + i)
            current_array.pop()
    result = []
    helper([], 0)
    return result

In [17]:
grid_size = 3 # assume 2 by 2 grid for now
G = list(it.product(range(grid_size), range(grid_size)))
ns = 4 # 4 distinct images
nG = len(G)
D = np.array([[manhattan_distance(tuple1, tuple2) for tuple2 in G] for tuple1 in G])
T = 40 # assume only 4 time steps for tractability

In [18]:
s = [create_random_array(),create_random_array()]

reward_assignment([np.ones(nG),np.ones(nG)])
reward_assignment([np.array([2,2,2,2,1,0,0,0,0]),np.array([0,0,0,0,1,2,2,2,2])])
reward_assignment([np.array([2,2,2,2,1,0,0,0,0]),np.array([2,2,2,2,1,0,0,0,0])])

AxisError: axis -2 is out of bounds for array of dimension 1

# Solo version MDP planing model with only 2 by 2 grid. Not tractable for even 3 by 3. 

In [ ]:
import mdptoolbox
from collections import Counter

grid_size = 2 # assume 2 by 2 grid for now
G = list(it.product(range(grid_size), range(grid_size)))
ns = 4 # 4 distinct images
nG = len(G)
D = np.array([[manhattan_distance(tuple1, tuple2) for tuple2 in G] for tuple1 in G])
T = 9 # assume only 4 time steps for tractability
N = grid_size**2 # similarly assume only 4 quanta per person
beta = 1
kappa = 0.01

assignments = np.unique([np.array([Counter(assignment)[g] for g in G]) for assignment in it.product(G + ['unassigned'], repeat=N)], axis=0)
states = list(it.product(assignments, range(nG), range(T)))
actions = list(range(nG))

def transition_function(s, a, s_next):
    f, g, t = s
    f_next, g_next, t_next = s_next

    if t == T-1 and (f == f_next).all() and g==g_next and t==t_next: # terminal states are absorbing
        return 1
    if t_next - t != 1 or g_next != a:
        #we require the next state to be the next time step and track the current action
        return 0
    if f[a] == N and (f == f_next).all(): # if all quanta are at a location, cannot add anymore.
        return 1
    
    diff_quanta = f_next - f
    if diff_quanta[a] != 1:
        #quantum +1 deterministically for the maintained grid
        return 0
    has_one_negative = []
    for g in range(nG):
        if g == a:
            continue
        if diff_quanta[g] != 0: 
            if len(has_one_negative) != 0: #if there is more than one location with different quanta amount
                return 0 #not a valid transition
            elif diff_quanta[g] != -1: #doesn't tolerate diff that's not 0 or -1
                return 0
            else: #if we see one -1, store it. If it's the only one it's permissible.
                has_one_negative.append(g)
    if not has_one_negative: # moving quanta from unassigned
        unassigned_quanta = N - np.sum([f[g] for g in range(nG)])
        return  unassigned_quanta / (N - f[a])
    else:
        assert len(has_one_negative) == 1
        return  f[has_one_negative[0]] / (N - f[a])

def reward_function(s, a, s_next):
    _, g, _ = s
    f_next, g_next, t_next = s_next
    d = D[g, a]
    if t_next != T-1:
        return -kappa*d
    else:
        
        return np.sum([quanta_to_prob(beta,f,ns) for f in f_next]) - kappa*d

transition_matricies, reward_matricies = [], []
for a in actions:
    trans_data = []
    rew_data = []
    row_indices = []
    col_indices = []
    nStates = len(states)
    for r in range(nStates):
        for c in range(nStates):
            p = transition_function(states[r], a,states[c])
            if p != 0:
                row_indices.append(r)
                col_indices.append(c)
                trans_data.append(p)
                rew_data.append(reward_function(states[r], a,states[c]))
    transition_matricies.append(csr_matrix((trans_data, (row_indices, col_indices)), shape=(nStates, nStates)))
    reward_matricies.append(csr_matrix((rew_data, (row_indices, col_indices)), shape=(nStates, nStates)))

MDP = mdptoolbox.mdp.FiniteHorizon(transition_matricies, reward_matricies, discount=1, N=T)
MDP.run()
MDP.V[:,:-1]
MDP.policy

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3]])